In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from plotting_functions import *
from analysis_functions import *
import cmocean
import matplotlib.colors as mcolors

from Additional_Colormaps import *
# load nature from json
with open('../misc/nature_reviews.json') as f:
    nature = json.load(f)

display = False
if display:
    display_colormaps([create_colormap('stone', nature['stone']),
                       create_colormap('grey', nature['grey']),
                       create_colormap('red', nature['red']),
                       create_colormap('blue', nature['blue']),
                       create_colormap('yellow', nature['yellow'])])

new_cmaps = load_all_cmaps()
domains = get_domain_bounds()

In [2]:
# ----------------------------------------------
# Flag for whether to mask out the ocean
# ----------------------------------------------

# - flag for whether to mask out the ocean
bool_mask_ocean = False
# - flag for whether to exclude ocean Hg0 uptake from 'Total_Hg_Deposition'
bool_exclude_ocean_Hg0_uptake = False

if bool_mask_ocean:
    area_var = 'land area [m2]'
else:
    area_var = 'total area [m2]'
# ----------------------------------------------

In [3]:
plt.style.use('../misc/acgc.mplstyle')

stones  = create_colormap('stone',  nature['stone'])
greys   = create_colormap('grey',   nature['grey'])
reds    = create_colormap('red',    nature['red'])
blues   = create_colormap('blue',   nature['blue'])
yellows = create_colormap('yellow', nature['yellow'])

# set axes.prop_cycle to custom color cycle
new_colors = [stones.colors[4], greys.colors[5]]#[blues.colors[3], reds.colors[3], yellows.colors[3], greys.colors[3], stones.colors[4]]
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=new_colors)

In [ ]:
baseline = xr.open_mfdataset('../model_output/v14.3/Baseline_deposition.nc')
IKU_fire_TGM = xr.open_mfdataset('../model_output/v14.3/Fire_TGM_deposition.nc')
IKU_fire_TPM = xr.open_mfdataset('../model_output/v14.3/Fire_TPM_deposition.nc')

# scale deposition fields by ratio between the emission magnitude used and the estimated emission magnitude
f_TGM = 0.935
scale_TGM = (556*f_TGM)/565.9
scale_TPM = (556*(1-f_TGM))/21.9
print('scale factors |', 'TGM:', np.round(scale_TGM,2), 'TPM:', np.round(scale_TPM,2))
var_list = [v for v in IKU_fire_TGM.data_vars if 'AREA' not in v]
IKU_fire_TGM[var_list] = IKU_fire_TGM[var_list]*scale_TGM
IKU_fire_TPM[var_list] = IKU_fire_TPM[var_list]*scale_TPM

IKU_fire_all = IKU_fire_TGM[['Total_Hg_Deposition', 'Total_Hg_Wet_Dep', 'Total_Hg_Dry_Dep']] + IKU_fire_TPM[['Total_Hg_Deposition', 'Total_Hg_Wet_Dep', 'Total_Hg_Dry_Dep']]
IKU_fire_all['AREA'] = IKU_fire_TGM['AREA']

baseline = baseline.sel(time=slice('2015-05-01','2016-05-01'))
IKU_fire_all = IKU_fire_all.sel(time=slice('2015-05-01','2016-05-01'))

# -- mask out ocean area
if bool_mask_ocean:
    # mask baseline and IKU_fire_all to only include land area
    masks = xr.open_mfdataset('../misc/masks.nc')
    baseline     = xr.merge([baseline, masks])
    IKU_fire_all = xr.merge([IKU_fire_all, masks])
    IKU_fire_TGM = xr.merge([IKU_fire_TGM, masks])
    IKU_fire_TPM = xr.merge([IKU_fire_TPM, masks])

    baseline     = baseline.where(baseline['Met_FROCEAN']>= 0.995, drop=True)
    IKU_fire_all = IKU_fire_all.where(IKU_fire_all['Met_FROCEAN']>= 0.995, drop=True)
    IKU_fire_TGM = IKU_fire_TGM.where(IKU_fire_TGM['Met_FROCEAN']>= 0.995, drop=True)
    IKU_fire_TPM = IKU_fire_TPM.where(IKU_fire_TPM['Met_FROCEAN']>= 0.995, drop=True)

# -- exclude ocean Hg0 uptake from 'Total_Hg_Deposition'
if bool_exclude_ocean_Hg0_uptake:
    # -- redefine 'Total_Hg_Deposition' to be the sum of 'Total_Hg_Wet_Dep' and 'Total_Hg_Dry_Dep'
    baseline['Total_Hg_Deposition']     = baseline['Total_Hg_Wet_Dep'] + baseline['Total_Hg_Dry_Dep']
    IKU_fire_all['Total_Hg_Deposition'] = IKU_fire_all['Total_Hg_Wet_Dep'] + IKU_fire_all['Total_Hg_Dry_Dep']
    IKU_fire_TGM['Total_Hg_Deposition'] = IKU_fire_TGM['Total_Hg_Wet_Dep'] + IKU_fire_TGM['Total_Hg_Dry_Dep']
    IKU_fire_TPM['Total_Hg_Deposition'] = IKU_fire_TPM['Total_Hg_Wet_Dep'] + IKU_fire_TPM['Total_Hg_Dry_Dep']

In [ ]:
def get_total_dep(ds, units='kg/mo'):
    """ Calculate total deposition for each grid cell and return a new dataset. """
    days_in_year  = 365.25
    if units in ['kg/mo', 'kg mo-1']:
    # convert from ug/m2/a to ug/m2
        days_in_month = ds.time.dt.days_in_month
        weights = days_in_month/days_in_year
    elif units in ['kg/yr', 'kg yr-1']:
        weights = 1
    elif units in ['kg/day', 'kg d-1']:
        weights = 1/days_in_year
    else:
        raise ValueError('units must be one of kg/mo, kg/yr, or kg/day')

    ds = ds['Total_Hg_Deposition']*ds['AREA']*1e-9
    ds = ds*(weights)
    ds = ds.sum(dim=['lat','lon'])
    return ds


ds1, label1, color1 = get_total_dep(IKU_fire_TGM), 'TGM', stones.colors[3]
ds2, label2, color2 = get_total_dep(IKU_fire_TPM), 'TPM', greys.colors[4]
ds3, label3, color3 = ds1+ds2, 'TGM+TPM', 'k'

# make bar plot
plt.figure(figsize=(4,2))
ax = plt.subplot(111)
x = np.arange(len(ds1.time))
ax.bar(x, ds1, color=color1, edgecolor='k', label=label1, zorder=2)
ax.bar(x, ds2, bottom=ds1, color=color2, edgecolor='k', label=label2, zorder=2)
ax.set_ylabel(r'Hg Deposition (kg mo$^{-1}$)')
ax.set_xticks(x)
yr_str = [str(yr.values.item()) for yr in ds1.time.dt.year]
mo_str = [str(mo.values.item()) for mo in ds1.time.dt.month]
ax.legend()
ax.set_xticklabels(['May', 'June', 'July', 'August', 'Sept.', 'Oct.', 'Nov.', 'Dec.', 'Jan.', 'Feb.', 'Mar.', 'April', ], rotation=0, ha='center');

ax.set_xlim(-0.5, 6.5)
ax.set_ylim(0, 65)
#plt.savefig('../figures/Alaska_domain/png/Total_deposition_bar_chart.png', format='png')
#plt.savefig('../figures/Alaska_domain/svg/Total_deposition_bar_chart.svg', format='svg')

In [ ]:
target_var = 'total deposition [kg]'

# display total deposition for each domain 
annual_dep = pd.read_csv('../tables/deposition_annual_v14_3.csv')

# - remove surface deposition scenario, as it is used as a comparison point but is not a primary scenario
annual_dep = annual_dep[annual_dep['fraction FT label']!='surface']

# now calculate f_TPM range
TGM = annual_dep[annual_dep['emission scenario']=='Fire (TGM)']
TPM = annual_dep[annual_dep['emission scenario']=='Fire (TPM)']
f_TPM = pd.merge(TGM, TPM, on=['domain name', 'magnitude label', 'fraction TGM label', 'fraction FT label', 'area [m2]'], suffixes=('_TGM', '_TPM'))
f_TPM.drop(columns=['emission scenario_TGM', 'emission scenario_TPM'], inplace=True)
f_TPM['f_TPM'] = (f_TPM[f'{target_var}_TPM']/(f_TPM[f'{target_var}_TGM']+f_TPM[f'{target_var}_TPM']))

# loop over each domain and print the total deposition for each emission scenario
# - show the `mid` scenario and the range of deposition across all other scenarios
for domain in annual_dep['domain name'].unique():
    print(f'{domain} domain:')
    print('----------------')
    for scenario in ['Fire (all)', 'Fire (TGM)', 'Fire (TPM)', 'f_TPM']:
        if scenario != 'f_TPM':
            subset = annual_dep[(annual_dep['domain name']==domain) & (annual_dep['emission scenario']==scenario)]
            mid = subset[ (subset['magnitude label']=='mid') & (subset['fraction TGM label']=='mid') & (subset['fraction FT label']=='mid')][target_var].values[0]
            min_ = np.min(subset[target_var])
            max_ = np.max(subset[target_var])
            print(f'{scenario}: {mid:.2f} [{min_:.2f} - {max_:.2f}] kg')
        else:
            subset = f_TPM[(f_TPM['domain name']==domain)]
            mid = subset[ (subset['magnitude label']=='mid') & (subset['fraction TGM label']=='mid') & (subset['fraction FT label']=='mid')]['f_TPM'].values[0]
            min_ = np.min(subset['f_TPM'])
            max_ = np.max(subset['f_TPM'])
            print(f'TPM fraction: {mid:.2f} [{min_:.2f} - {max_:.2f}]')
    print('')


In [ ]:
df = pd.read_csv('../tables/deposition_table_v14_3.csv')

print('Fraction of fire-attributable deposition occurring during june and july:')
print('---')
# get the fraction of deposition occurring during june and july
for domain in ['Alaska','YKD','IKU']:
    print(f'{domain} domain:')
    print('----------------')
    for scenario in ['Fire (TGM)', 'Fire (TPM)', 'Fire (all)']:
        subset = df[(df['domain name']==domain) & (df['emission scenario']==scenario)]

        grouping_vars = ['emission scenario', 'domain name', 'magnitude label', 'fraction TGM label', 'fraction FT label']
        first_vars = ['year', 'latitude min', 'latitude max', 'longitude min', 'longitude max', 'area [m2]', 'emission [kg]']
        sum_vars = ['total deposition [kg/mo]', 'total deposition [ug/m2/mo]']
        agg_dict = {var:'sum' for var in sum_vars}
        agg_dict.update({var:'first' for var in first_vars})

        # -- calculate total deposition for each scenario
        subset_all = subset.groupby(by=grouping_vars, as_index=False).agg(agg_dict)
        subset_all.rename(columns={'total deposition [kg/mo]':'total deposition [kg]',}, inplace=True)

        # -- calculate total deposition for each scenario
        subset_june_july = subset[(subset['year']==2015) & (subset['month'].isin([6,7]))]
        subset_june_july = subset_june_july.groupby(by=grouping_vars, as_index=False).agg(agg_dict)
        subset_june_july.rename(columns={'total deposition [kg/mo]':'total deposition [kg]',}, inplace=True)

        # -- merge june_july in with all using the same grouping variables
        subset_all = pd.merge(subset_all, subset_june_july[grouping_vars+['total deposition [kg]']], on=grouping_vars, suffixes=('_all', '_june_july'))
        
        # -- calculate the fraction of deposition that occurs during june and july
        subset_all['f_june_july'] = subset_all['total deposition [kg]_june_july']/subset_all['total deposition [kg]_all']

        mid = subset_all[ (subset_all['magnitude label']=='mid') & (subset_all['fraction TGM label']=='mid') & (subset_all['fraction FT label']=='mid')]['f_june_july'].values[0]
        min_ = np.min(subset_all['f_june_july'])
        max_ = np.max(subset_all['f_june_july'])

        print(f'{scenario}: {mid:.2f} [{min_:.2f} - {max_:.2f}]')
    print('')

In [ ]:
df = pd.read_csv('../tables/deposition_table_v14_3.csv')

for month_list in [[6,7], df['month'].unique().tolist()]:

    if month_list == [6,7]:
        print('Fraction of total deposition from fire-attributable sources during active fire period (june and july, 2015):')
    elif month_list == df['month'].unique().tolist():
        print('Fraction of total deposition from fire-attributable sources during the entire year (May 2015 - May 2016):')

    print('---')
    # get the fraction of deposition occurring during june and july
    for domain in ['Alaska','YKD','IKU']:
        baseline = df[(df['domain name']==domain) & (df['emission scenario']=='Baseline') & (df['year']==2015) & (df['month'].isin(month_list))]['total deposition [kg/mo]'].sum()

        for scenario in ['Fire (all)', ]:
            subset = df[(df['domain name']==domain) & (df['emission scenario']==scenario)]

            subset = subset[(subset['year']==2015) & (subset['month'].isin(month_list))]
            grouping_vars = ['emission scenario', 'domain name', 'magnitude label', 'fraction TGM label', 'fraction FT label']
            first_vars = ['year', 'latitude min', 'latitude max', 'longitude min', 'longitude max', 'area [m2]', 'emission [kg]']
            sum_vars = ['total deposition [kg/mo]', 'total deposition [ug/m2/mo]']
            agg_dict = {var:'sum' for var in sum_vars}
            agg_dict.update({var:'first' for var in first_vars})

            # -- calculate time subset deposition for each scenario
            subset_time = subset.groupby(by=grouping_vars, as_index=False).agg(agg_dict)
            subset_time.rename(columns={'total deposition [kg/mo]':'total deposition [kg]',}, inplace=True)

            mid = subset_time[ (subset_time['magnitude label']=='mid') & (subset_time['fraction TGM label']=='mid') & (subset_time['fraction FT label']=='mid')]['total deposition [kg]'].values[0]
            min_ = np.min(subset_time['total deposition [kg]'])
            max_ = np.max(subset_time['total deposition [kg]'])

            f_fire_mid = mid/(mid+baseline)
            f_fire_min = min_/(min_+baseline)
            f_fire_max = max_/(max_+baseline)

            print(f'[{domain} domain] {scenario}: {f_fire_mid:.2f} [{f_fire_min:.2f} - {f_fire_max:.2f}]')
    print('')


In [9]:
def stack_treatments(df, var:str, domain:str, emission_scenario:str):
    ''' Stack the treatments for a given domain and emission scenario. 
        Used to compute max and min values for the fire-attributable fraction of deposition. 
        
        Parameters:
        -----------
        df: pd.DataFrame
            DataFrame containing the deposition data.
        var: str
            The variable to stack.
        domain: str
            The domain name.
        emission_scenario: str
            The emission scenario.
        
        Returns:
        --------
        stacked: np.array
            The stacked array.
        '''
    stacked = []
    for magnitude_label in ['low','mid','high']:
        for fraction_TGM_label in ['low','mid','high']:
            for fraction_FT_label in ['low','mid','high']:
                subset = df[(df['domain name']==domain) & (df['emission scenario']==emission_scenario) & (df['magnitude label']==magnitude_label) & (df['fraction TGM label']==fraction_TGM_label) & (df['fraction FT label']==fraction_FT_label)]
                stacked.append(subset[var].values)
    return np.array(stacked)

#stack_treatments(df, 'total deposition [kg/mo]', 'Alaska', 'Fire (all)').mean(axis=0)


In [ ]:
# - set ticklabel size
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
# set defult marker size
plt.rcParams['lines.markersize'] = 5.6
markeredgewidth=0.8

for v, vlabel, fn in zip(['total deposition [kg/mo]', 'total deposition [ug/m2/mo]'], 
                         ['Hg deposition (kg mo$^{-1}$)', 'Hg deposition ($\mathrm{\mu}$g m$^{-2}$ mo$^{-1}$)'], 
                         ['monthly_deposition_kg_mo', 'monthly_deposition_ug_m2_mo']):

    fig = plt.figure(figsize=(3.25,2.45))

    for plt_n, domain in enumerate(['Alaska','YKD','IKU']):
        ax = plt.subplot(3,1,plt_n+1)

        for scenario in ['Baseline']:
            subset = df[(df['domain name']==domain) & (df['emission scenario']==scenario)]
            x = pd.to_datetime(subset['date'].values)
            ax.plot(np.arange(len(x)), subset[v], marker='o', markeredgecolor='w', markeredgewidth=markeredgewidth)
            ax.text(0.98, 0.95, f'{domain}', va='top', ha='right', transform=ax.transAxes, fontsize=10, color='0.2', weight='regular', ) #bbox=dict(facecolor='white', edgecolor='white', alpha=0.5))
            ax.set_xticks(ticks = np.arange(len(x)))
            ax.axvline(x=8, color='0.2', lw=0.5, ls='-', zorder=0)
            xmin, xmax = ax.get_xlim()
            f_x = (8)/(xmax)
            if domain == 'Alaska':
                ax.text(f_x+0.03, 0.02, r'2016 $\rightarrow$', va='bottom', ha='left', transform=ax.transAxes, 
                        fontsize=8, color='0.2', weight='regular')
                ax.text(f_x-0.0, 0.02, r'$\leftarrow$ 2015', va='bottom', ha='right', transform=ax.transAxes, 
                        fontsize=8, color='0.2', weight='regular')

        # now add the fire scenario
        for scenario in ['Fire (all)']:
            subset = df[(df['domain name']==domain) & (df['emission scenario']==scenario)]
            mid = subset[ (subset['magnitude label']=='mid') & (subset['fraction TGM label']=='mid') & (subset['fraction FT label']=='mid')]
            # get the max and min values for the fire-attributable fraction of deposition
            stacked = stack_treatments(df, v, domain, scenario)
            min_ = stacked.min(axis=0)
            max_ = stacked.max(axis=0)

            x_fire = pd.to_datetime(mid['date'].values)
            y_fire = np.array(mid[v].values)

            min_[min_ == 0.] = np.nan
            max_[max_ == 0.] = np.nan
            y_fire[y_fire == 0.] = np.nan

            area = subset['area [m2]'].values[0]

            ax.plot(np.arange(len(y_fire)), y_fire, marker='o', markeredgecolor='w', markeredgewidth=markeredgewidth)
            ax.fill_between(np.arange(len(min_)), min_, max_, facecolor=greys.colors[5], edgecolor='None', alpha=0.2)

            if plt_n == 0:
                ax.text(0.02, 0.65, 'background', va='top', ha='left', transform=ax.transAxes, fontsize=8, color=new_colors[0], weight='regular', )
                ax.text(0.05, 0.3, 'fire-attributable', va='top', ha='left', transform=ax.transAxes, fontsize=8, color=new_colors[1], weight='regular', )

        # expand y-axis limits by 5%
        ymin, ymax = ax.get_ylim()
        ax.set_ylim(ymin, ymax+(ymax*0.05))

        ax.set_xticklabels([])
        # remove minor ticks
        ax.tick_params(axis='x', which='minor', bottom=False)
        # manually make gridlines white for now  add minor grid
        ax.grid(which='minor', axis='y', lw=0.5, ls='-', color='w', alpha=0)
        ax.grid(which='major', axis='y', lw=0.5, ls='-', color='w', alpha=0)
        ax.grid(which='major', axis='x', lw=0.5, ls='-', color='w', alpha=0)
        ax.grid(which='minor', axis='x', lw=0.5, ls='-', color='w', alpha=0)

        # add second y-axis with deposition in ug/m2/mo if using kg/mo
        if v == 'total deposition [kg/mo]':
            ax2 = ax.twinx()
            ymin, ymax = ax.get_ylim()
            ymin2, ymax2 = (ymin*1e9)/area, (ymax*1e9)/area
            ax2.set_ylim(ymin2, ymax2)
            ax2.grid(False)
            if plt_n == 0:
                ax2.set_yticks(ticks = [0, 1])
                ax2.set_yticks(ticks = [0.5, 1.5], minor=True)                
            if plt_n == 1:
                ax.set_ylabel(vlabel, fontsize=9);
                ax2.set_ylabel('Hg deposition ($\mathrm{\mu}$g m$^{-2}$ mo$^{-1}$)', labelpad=14, fontsize=9, rotation=270)
                ax2.set_yticks(ticks = [0, 1])
                ax2.set_yticks(ticks = [0.5, 1.5], minor=True)
            if plt_n == 2:
                ax2.set_yticks(ticks = [0, 2, 4])
                ax2.set_yticks(ticks = [1, 3, 5], minor=True)
        
        if v == 'total deposition [ug/m2/mo]':
            if plt_n == 0:
                ax.set_yticks(ticks = [0, 1])
                ax.set_yticks(ticks = [0.5, 1.5], minor=True)
            if plt_n == 1:
                ax.set_yticks(ticks = [0, 1])
                ax.set_yticks(ticks = [0.5, 1.5], minor=True)
            if plt_n == 2:
                ax.set_yticks(ticks = [0, 2, 4])
                ax.set_yticks(ticks = [1, 3, 5], minor=True)

            ax2 = ax.twinx()
            ymin, ymax = ax.get_ylim()
            ymin2, ymax2 = (ymin/1e9)*area, (ymax/1e9)*area
            ax2.set_ylim(ymin2, ymax2)
            ax2.grid(False)
            if plt_n == 1:
                ax.set_ylabel(vlabel, fontsize=9);
                ax2.set_ylabel('Hg deposition (kg mo$^{-1}$)', labelpad=18, fontsize=9, rotation=270)
    
    xlabels = [5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4]
    ax.set_xticks(ticks = np.arange(len(x)), labels = xlabels, fontsize=8);
    ax.set_xlabel('Month', fontsize=9);

    plt.savefig(f'../figures/misc/{fn}_wide.pdf', format='pdf')
    plt.savefig(f'../figures/misc/{fn}_wide.png', dpi=2000)


In [ ]:
# --- calculate the fraction of speciated emissions that deposits within the domain
df = pd.read_csv('../tables/deposition_annual_v14_3.csv')
df['f_redeposit'] = df['total deposition [kg]']/df['emission [kg]']

for domain in ['Alaska','YKD','IKU']:
    for scenario in ['Fire (TGM)', 'Fire (TPM)']:
        subset = df[(df['domain name']==domain) & (df['emission scenario']==scenario)]
        mid = subset[ (subset['magnitude label']=='mid') & (subset['fraction TGM label']=='mid') & (subset['fraction FT label']=='mid')]['f_redeposit'].values[0]
        min_ = np.min(subset['f_redeposit'])
        max_ = np.max(subset['f_redeposit'])
        print(f'{domain} domain, {scenario}: {mid:.2f} [{min_:.2f} - {max_:.2f}]')

In [ ]:
# -- calculate the fire-attributable fraction of total deposition during June - July, 2015
df = pd.read_csv('../tables/deposition_table_v14_3.csv')
df = df[ (df['year']==2015) & (df['month'].isin([6,7])) ]
baselines = df[(df['emission scenario']=='Baseline')]#.values[0]
baselines = baselines.groupby(by=['domain name'], as_index=False).sum(numeric_only=True)
df = df.groupby(by=['emission scenario', 'domain name', 'magnitude label', 'fraction TGM label', 'fraction FT label'], as_index=False).sum(numeric_only=True)

# -- display fraction of fire-attributable deposition during june and july
for domain in ['IKU', 'YKD', 'Alaska']:
    base = baselines[baselines['domain name']==domain]['total deposition [kg/mo]'].values[0]
    central = df[(df['domain name']==domain) & (df['emission scenario']=='Fire (all)') & (df['magnitude label']=='mid') & (df['fraction TGM label']=='mid') & (df['fraction FT label']=='mid')]['total deposition [kg/mo]'].values[0]
    central = central/(central+base)

    fire = df[(df['domain name']==domain) & (df['emission scenario']=='Fire (all)')]['total deposition [kg/mo]']
    fractions = fire/(fire+base)
    print('central:', np.round(central, 3), 'lower:', np.round(fractions.min(), 3), 'upper:', np.round(fractions.max(),3))

In [ ]:
AK_deps = df[(df['domain name']=='Alaska') & (df['emission scenario']=='Fire (all)')].groupby(by=['emission scenario', 'domain name', 'magnitude label', 'fraction TGM label', 'fraction FT label'], as_index=False).sum()
central = AK_deps[(AK_deps['magnitude label']=='mid') & (AK_deps['fraction TGM label']=='mid') & (AK_deps['fraction FT label']=='mid')]['total deposition [kg/mo]'].values[0]
AK_deps = AK_deps['total deposition [kg/mo]']
print('Alaska:', np.round(central, 2), '(', np.round(AK_deps.min(),2), '-', np.round(AK_deps.max(),2), ')')